* slew performance
* delay to on-sky
* loss of time at end of night
* faults within the night
* weather downtime
* scheduled downtime

Relating these to overall system availability:

For dayobs 20251108
* total time in the night = 8.06 hours
* time lost from -12 deg twilight to start of science = 1.2 hours (includes BLOCK-T539)
* time lost due to faults and recovery ~2.44 hours
* time lost current slew speed compared to baseline_v5.1.0  = 0.5 hours
* time slewing/changing filter = 1.4 hours
* time that would be spent slewing/changing filter if slew like v5.1. = 0.87 hours
* time spent taking image = 2.9 hours
* time exposing at sky = 2.82  (0.094 hours shutter movement)
* science open shutter fraction 20251108 = 35%
* median open shutter fraction in v5.1 = 75%

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib widget

In [ ]:
from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site

In [ ]:
from lsst.summit.utils import ConsDbClient

client = ConsDbClient("http://consdb-pq.consdb:8080/consdb")

In [ ]:
instrument = 'lsstcam'

In [ ]:
def fetch(day_obs_min, day_obs_max):
    visits_query = f'''
    SELECT 
        *
    FROM
        cdb_{instrument}.visit1 AS visit1 
    WHERE
        airmass > 0.
        AND img_type in ('acq', 'science', 'cwfs', 'focus')
        AND day_obs >= {day_obs_min} AND day_obs <= {day_obs_max}
    '''

    visits = client.query(visits_query).to_pandas().sort_values(by='obs_start_mjd')

    return visits

In [ ]:
day_obs_min = 20251026
day_obs_max = 20260107
visits = fetch(day_obs_min, day_obs_max)

In [ ]:
visits

In [ ]:
visits['day_obs'].value_counts().sort_index()

In [ ]:
np.unique(visits['img_type'])

In [ ]:
np.sort(visits.columns)

In [ ]:
visits[selection_day_obs]

In [ ]:
from astropy.time import Time

In [ ]:
def dayObsToTime(day_obs):
    year = str(day_obs)[0:4]
    month = str(day_obs)[4:6]
    day = str(day_obs)[6:8]
    time = Time('%s-%s-%s 00:00:00'%(year, month, day), format='iso')
    return time

In [ ]:
def dayObsToTwilight(day_obs, observer):
    year = str(day_obs)[0:4]
    month = str(day_obs)[4:6]
    day = str(day_obs)[6:8]
    time = Time('%s-%s-%s 23:59:00'%(year, month, day), scale='utc', format='iso')
    
    time_midnight = observer.midnight(time, which='nearest')
    time_evening_astronomical_twilight = observer.twilight_evening_astronomical(time_midnight, which='previous')
    time_morning_astronomical_twilight = observer.twilight_morning_astronomical(time_midnight, which='next')
    time_evening_nautical_twilight = observer.twilight_evening_nautical(time_midnight, which='previous')
    time_morning_nautical_twilight = observer.twilight_morning_nautical(time_midnight, which='next')
    return time_evening_astronomical_twilight, time_morning_astronomical_twilight, time_evening_nautical_twilight, time_morning_nautical_twilight

In [ ]:
from astroplan import Observer
observer = Observer.at_site('LSST')

In [ ]:
import astropy.units as u

In [ ]:
def convertTime(time_input, time_reference):
    time_output = (time_input - (time_reference + (1. * u.day))) * u.day.to(u.hr)
    return time_output.value

In [ ]:
def gapTime(time_exp_midpt_mjd, time_evening_astronomical_twilight, time_morning_astronomical_twilight, time_day_obs):
    selection_night = (time_exp_midpt_mjd > convertTime(time_evening_astronomical_twilight, time_day_obs)) & (time_exp_midpt_mjd < convertTime(time_morning_astronomical_twilight, time_day_obs))

    time_exp_midpt_mjd_augmented = np.insert(time_exp_midpt_mjd[selection_night], 0, convertTime(time_evening_astronomical_twilight, time_day_obs))
    time_exp_midpt_mjd_augmented = np.append(time_exp_midpt_mjd_augmented, convertTime(time_morning_astronomical_twilight, time_day_obs))

    assert(len(time_exp_midpt_mjd[selection_night]) + 2 == len(time_exp_midpt_mjd_augmented))
    assert(np.all(np.diff(time_exp_midpt_mjd_augmented) >= 0))

    selection_gap = np.diff(time_exp_midpt_mjd_augmented) * u.hr.to(u.min) > 5

    return np.sum(np.diff(time_exp_midpt_mjd_augmented)[selection_gap])

In [ ]:
time_exp_midpt_mjd = Time(visits['exp_midpt_mjd'], format='mjd')

In [ ]:
plt.figure(figsize=(12,8))

xticks = [dayObsToTime(day_obs).mjd for day_obs in np.unique(visits['day_obs'])]
xtick_labels = np.unique(visits['day_obs'])

day_obs_summary = {
    'day_obs': xtick_labels,
    'day_obs_mjd': xticks,
    'gap_time': [],
    'night_time': [],
}

for day_obs in np.unique(visits['day_obs']):

    time_day_obs = dayObsToTime(day_obs)
    time_evening_astronomical_twilight, time_morning_astronomical_twilight, time_evening_nautical_twilight, time_morning_nautical_twilight = dayObsToTwilight(day_obs, observer)

    selection_day_obs = (visits['day_obs'] == day_obs)
    fbs_programs = ['BLOCK-407', 'BLOCK-408', 'BLOCK-416', 'BLOCK-417', 'BLOCK-419']
    aos_programs = ['BLOCK-T539']

    time_exp_midpt_mjd = convertTime(
        Time(visits['exp_midpt_mjd'][selection_day_obs], format='mjd'), 
        time_day_obs
    )
    x = np.tile(time_day_obs.mjd, len(visits[selection_day_obs]))

    selection_fbs = np.isin(visits[selection_day_obs]['science_program'], fbs_programs)
    selection_aos = np.isin(visits[selection_day_obs]['science_program'], aos_programs)

    plt.scatter(x[~selection_fbs], time_exp_midpt_mjd[~selection_fbs], marker='_', c='tab:orange')
    plt.scatter(x[selection_fbs], time_exp_midpt_mjd[selection_fbs], marker='_', c='tab:blue')
    plt.scatter(x[selection_aos], time_exp_midpt_mjd[selection_aos], marker='_', c='tab:green')

    plt.scatter(time_day_obs.mjd, convertTime(time_evening_astronomical_twilight, time_day_obs), marker='o', c='black')
    plt.scatter(time_day_obs.mjd, convertTime(time_morning_astronomical_twilight, time_day_obs), marker='o', c='black')
    plt.scatter(time_day_obs.mjd, convertTime(time_evening_nautical_twilight, time_day_obs), marker='o', c='0.5')
    plt.scatter(time_day_obs.mjd, convertTime(time_morning_nautical_twilight, time_day_obs), marker='o', c='0.5')
    #plt.plot(time_day_obs.mjd, convertTime(time_evening_astronomical_twilight, time_day_obs))

    day_obs_summary['gap_time'].append(gapTime(time_exp_midpt_mjd, time_evening_astronomical_twilight, time_morning_astronomical_twilight, time_day_obs))
    day_obs_summary['night_time'].append((time_morning_astronomical_twilight - time_evening_astronomical_twilight).value * u.day.to(u.hr))

plt.ylim(-1., 9.5)
plt.xticks(xticks, xtick_labels, rotation=90.)
plt.ylabel('Time (hrs)')
plt.tight_layout()
plt.show()

for key in day_obs_summary.keys():
    day_obs_summary[key] = np.array([_ for _ in day_obs_summary[key]])

In [ ]:
gapTime(time_exp_midpt_mjd, time_evening_astronomical_twilight, time_morning_astronomical_twilight, time_day_obs)

In [ ]:
time_evening_astronomical_twilight.mjd

In [ ]:
(time_morning_astronomical_twilight - time_evening_astronomical_twilight).value * u.day.to(u.hr)

In [ ]:
day_obs_summary

In [ ]:
plt.figure()
plt.scatter(day_obs_summary['day_obs_mjd'], 1. - (day_obs_summary['gap_time'] / day_obs_summary['night_time']))
#plt.grid()
plt.ylim(0., 1.)
plt.xticks(xticks, xtick_labels, rotation=90.)
#plt.ylabel('Fraction Inactive\n(sum of visit gaps > 5 min)')
plt.ylabel('System Availability\n(visit gaps < 5 min)')
plt.tight_layout()

In [ ]:
plt.figure()
plt.hist(day_obs_summary['gap_time'] / day_obs_summary['night_time'], bins=np.linspace(0., 1., 21))

In [ ]:
np.mean(day_obs_summary['gap_time'] / day_obs_summary['night_time'])

In [ ]:
for _ in 

# Visit Gap Time Distribution

In [ ]:
fbs_programs = ['BLOCK-407', 'BLOCK-408', 'BLOCK-416', 'BLOCK-417', 'BLOCK-419']
selection_fbs = np.isin(visits['science_program'], fbs_programs)

In [ ]:
visit_gaps = pd.DataFrame({
    'day_obs': visits[selection_fbs]['day_obs'][1:],
    'visit_gap': (visits[selection_fbs]['obs_start_mjd'][1:].values - visits[selection_fbs]['obs_end_mjd'][0:-1].values) * u.day.to(u.second)
})


groups = visit_gaps.groupby('day_obs')
visit_gaps_summary = pd.DataFrame({
    'day_obs': groups['day_obs'].first(),
    'n_visits': groups['day_obs'].count(),
    'visit_gap_10': groups['visit_gap'].quantile(0.1),
    'visit_gap_50': groups['visit_gap'].quantile(0.50),
    'visit_gap_90': groups['visit_gap'].quantile(0.9),
})

In [ ]:
plt.figure()
plt.scatter(visit_gaps_summary['day_obs'], visit_gaps_summary['visit_gap_50'])
plt.ylim(0., 30.)

In [ ]:
bins = np.linspace(0., 60., 31)

selection_day_obs = (visit_gaps['day_obs'] >= 20251203)

plt.figure()
plt.hist(visit_gaps['visit_gap'][~selection_day_obs], bins=bins, histtype='step', density=True, label='day_obs < 2025 Dec 03')
plt.hist(visit_gaps['visit_gap'][selection_day_obs], bins=bins, histtype='step', density=True, label='day_obs >= 2025 Dec 03')
plt.xlim(0., 60.)
plt.legend()
plt.xlabel('Visit Gap (second)')
plt.ylabel('PDF')
plt.title('FBS pre-LSST Visits')